# Computer vision - build a neural network

*The content of this notebook is based on the official TensorFlow tutorial "TensorFlow 2 quickstart for beginners". Here is the original source code:*

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

## Data preparation

In [3]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Neural network design

Build the `tf.keras.Sequential` model by stacking layers. The ``Dropout`` layer randomly sets input units to 0 with a frequency of rate 0.2 at each step during training time, which helps prevent overfitting. 

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

### Optimizer and loss function

Next, we need to choose an **optimizer** and **loss** function for training:

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.16821931,  0.3898082 ,  0.12535964, -0.7232452 ,  0.51362497,
         0.00426391, -0.10888806, -0.35102203, -0.43187338, -0.5077769 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.12065846, 0.15058877, 0.11559633, 0.04947658, 0.17043765,
        0.10241248, 0.09145587, 0.07178838, 0.06621262, 0.06137285]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [10]:
loss_fn(y_train[:1], predictions).numpy()

2.2787466

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Model training

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 642us/step - loss: 0.4928 - accuracy: 0.8556
Epoch 2/5
1875/1875 [==============================] - 1s 635us/step - loss: 0.1537 - accuracy: 0.9536
Epoch 3/5
1875/1875 [==============================] - 1s 765us/step - loss: 0.1104 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 1s 703us/step - loss: 0.0866 - accuracy: 0.9743
Epoch 5/5
1875/1875 [==============================] - 1s 692us/step - loss: 0.0755 - accuracy: 0.9767


## Model evaluation

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [15]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0732 - accuracy: 0.9780


[0.07321056723594666, 0.9779999852180481]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [16]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.09392867e-08, 1.62570846e-09, 1.71217107e-06, 2.22564777e-05,
        2.99183530e-13, 4.48554367e-08, 1.24438166e-12, 9.99974966e-01,
        1.77110863e-07, 7.48243963e-07],
       [1.52942015e-09, 8.80617663e-05, 9.99895930e-01, 9.36737433e-06,
        4.43583938e-14, 1.88387526e-06, 7.11530346e-09, 4.60684968e-16,
        4.78148104e-06, 3.28602510e-12],
       [3.18908491e-07, 9.97908831e-01, 1.84463002e-04, 2.05156794e-05,
        1.49538915e-04, 1.29564996e-05, 4.39937212e-05, 7.54863955e-04,
        9.18019505e-04, 6.59599073e-06],
       [9.99574482e-01, 1.91707514e-07, 1.81093463e-04, 9.37698587e-07,
        4.54466817e-06, 1.49871175e-05, 2.02364303e-04, 1.57091508e-05,
        1.03851740e-07, 5.66688004e-06],
       [8.74082070e-06, 2.73454537e-10, 1.69499708e-05, 3.78712201e-07,
        9.79025066e-01, 3.72799059e-06, 1.73404951e-05, 1.17754782e-04,
        5.50070354e-06, 2.08045002e-02]], dtype=float32)>